# Постановка экспериментов на воспроизводимость модели

Загрузка нужных библиотек для работы

In [5]:
import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import  OneHotEncoder, LabelEncoder

In [7]:
# Загрузка сохраненной модели из файла pickle
with open("/content/drive/MyDrive/rand_forest_regressor.pkl", "rb") as f:
    model_for_work = pickle.load(f)

In [27]:
# создадим тестовый набор
data = [
    ('Pending',	2.0,True,	'Brook Park',	936.0,'44142','OH',	109900,	'single_family',	False,	'1958',	True,	True,	False,	4.4,	0.70),
    ('Active',	2.0,True,	'Fresno',	1650.0,	'93706','CA',	1100000,	'single_family',	False,	'1931',	False,	False,	False, 4.0,	0.87),
    ('Active', 5.0, False,'Dallas', 2300.0, '75205','TX', 2395000, 'single_family', False, '2006', True, True, True, 6.7, 1.19),
    ('Active	',3.0,False,'Kansas',	2058.0,	'64112','MO',	790000,	'condos',	False,	'2008',	True,	True,	False,	2.3,	2.70),
    ('Active', 4.0, False, 'Palm Bay', 2990.0, '75205','FL', 2395000, 'single_family', False, '2006', True, True, True, 6.7, 1.19)
]

columns = ['status','baths','fireplace','city','sqft','zipcode','state','target','Type','pool','Year built','Heating','Cooling','Parking','school_rating _mean', 'school_dist_min']

df_pred = pd.DataFrame(data, columns=columns)

df_pred.head()

,status,baths,fireplace,city,sqft,zipcode,state,target,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
0,Pending,2.0,True,Brook Park,936.0,44142,OH,109900,single_family,False,1958,True,True,False,4.4,0.70
1,Active,2.0,True,Fresno,1650.0,93706,CA,1100000,single_family,False,1931,False,False,False,4.0,0.87
2,Active,5.0,False,Dallas,2300.0,75205,TX,2395000,single_family,False,2006,True,True,True,6.7,1.19
3,Active\t,3.0,False,Kansas,2058.0,64112,MO,790000,condos,False,2008,True,True,False,2.3,2.70
4,Active,4.0,False,Palm Bay,2990.0,75205,FL,2395000,single_family,False,2006,True,True,True,6.7,1.19


In [55]:
def preprocessing_for_predict(data, model):

    def processing_data(data):
      df_out = data.copy()
      # Нормализация и логарифмирование данных
      for column in ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']:
        df_out[column] = df_out[column].apply(lambda x: abs(x))
        constant = 1e-6
        df_out[column] = np.log(df_out[column] + constant)
    #One-Hot Encoding (OHE) для категориальных признаков
      ohe_status = OneHotEncoder(sparse=False)
      ohe_state = OneHotEncoder(sparse=False)
      ohe_Type = OneHotEncoder(sparse=False)

      status_ohe = ohe_status.fit_transform(df_out['status'].values.reshape(-1,1))
      state_ohe = ohe_state.fit_transform(df_out['state'].values.reshape(-1,1))
      Type_ohe = ohe_Type.fit_transform(df_out['Type'].values.reshape(-1,1))

    #Label Encoding (LE) для категориальных признаков:
      le = LabelEncoder()
      state_label = le.fit_transform(df_out['state'])

      year_le = LabelEncoder()
      year_ord = year_le.fit_transform(df_out['Year built'])

      city_le = LabelEncoder()
      city_label = city_le.fit_transform(df_out['city'])

      zip_le = LabelEncoder()
      zip_label = zip_le.fit_transform(df_out['zipcode'])

    # Добавление закодированных категориальных признаков
      df_out = df_out.join(pd.DataFrame(status_ohe, columns=['status_' + str(cat) for cat in ohe_status.categories_[0]]))
      df_out = df_out.join(pd.DataFrame(state_ohe, columns=['state_' + str(cat) for cat in ohe_state.categories_[0]]))
      df_out = df_out.join(pd.DataFrame(Type_ohe, columns=['Type_' + str(cat) for cat in ohe_Type.categories_[0]]))
      df_out['state_label'] = state_label
      df_out['year_ord'] = year_ord
      df_out['city_label'] = city_label
      df_out['zip_label'] = zip_label

    #Удаление исходных категориальных столбцов:
      df_out.drop(['status', 'state', 'Type', 'city', 'zipcode','Year built'], axis=1, inplace=True)

      return df_out

    X_test = processing_data(data)
    y_test_pred_loaded = model.predict(X_test)
    target = np.exp(y_test_pred_loaded)
    rounded_target = np.round(target)
    print(rounded_target)

    return rounded_target


In [56]:
m_predictions = preprocessing_for_predict(df_pred, model_for_work)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- status_Active	
- target
Feature names seen at fit time, yet now missing:
- Type_apartment
- Type_contemporary 
- Type_historical
- Type_land
- Type_mobile_home
- ...


In [54]:
print(m_predictions)

None
